In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit
from pyspark.sql.types import *
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [2]:
# Read lines from the text file
answers_sdf = spark.read.load('sx-stackoverflow-a2q.txt', format="text")
comments_answers_sdf = spark.read.load('sx-stackoverflow-c2a.txt', format="text")
comments_questions_sdf = spark.read.load('sx-stackoverflow-c2q.txt', format="text")

## Step 2.2

In [3]:
answers_sdf.count()

17823525

In [4]:
answers_sdf.show(10)

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
+----------------+
only showing top 10 rows



In [5]:
answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
comments_answers_sdf.count()

25405374

In [7]:
comments_answers_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
+--------------------+
only showing top 10 rows



In [8]:
comments_answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
comments_questions_sdf.count()

20268151

In [10]:
comments_questions_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|4550 4550 1220729190|
|  242 184 1220733503|
|4213 4946 1220768149|
|    91 91 1220768295|
|2658 1874 1220771891|
|4035 1874 1220773037|
|2257 4489 1220802041|
|  577 577 1220834891|
|4489 4489 1220853536|
| 828 2783 1220854143|
+--------------------+
only showing top 10 rows



In [11]:
comments_questions_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
answers_sdf.createOrReplaceTempView('answers_sdf_view')
answers_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , FROM_UNIXTIME(split(value, " ")[2]) AS time_stamp,"answer" AS edge_type FROM answers_sdf_view')

comments_answers_sdf.createOrReplaceTempView('comments_answers_sdf_view')
comments_answers_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , FROM_UNIXTIME(split(value, " ")[2]) AS time_stamp,"comment-on-answer" AS edge_type FROM comments_answers_sdf_view')

comments_questions_sdf.createOrReplaceTempView('comments_questions_sdf_view')
comments_questions_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , FROM_UNIXTIME((split(value, " ")[2])) AS time_stamp, "comment-on-question" AS edge_type FROM comments_questions_sdf_view')

In [13]:
#graph_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , CAST(split(value, " ")[2] AS string) AS from_unixtime,"answer" AS edge_type FROM answers_sdf_view')
#graph_sdf.unionAll(comments_answers_sdf).unionAll(comments_questions_sdf)
graph_sdf=answers_sdf.union(comments_answers_sdf).union(comments_questions_sdf)
graph_sdf.count()

63497050

## Step 2.3

In [14]:
answers_sdf.count()

17823525

In [15]:
answers_sdf.show(5)

+---------+-------+-------------------+---------+
|from_node|to_node|         time_stamp|edge_type|
+---------+-------+-------------------+---------+
|        9|      8|2008-08-01 05:17:57|   answer|
|        1|      1|2008-08-01 06:56:41|   answer|
|       13|      1|2008-08-01 15:57:27|   answer|
|       17|      1|2008-08-01 19:07:19|   answer|
|       48|      2|2008-08-01 19:16:22|   answer|
+---------+-------+-------------------+---------+
only showing top 5 rows



In [16]:
answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [17]:
comments_answers_sdf.count()

25405374

In [18]:
comments_answers_sdf.show(5)

+---------+-------+-------------------+-----------------+
|from_node|to_node|         time_stamp|        edge_type|
+---------+-------+-------------------+-----------------+
|        1|     91|2008-09-06 15:07:10|comment-on-answer|
|        3|     91|2008-09-06 15:09:52|comment-on-answer|
|      380|    350|2008-09-06 15:42:16|comment-on-answer|
|     4642|   2257|2008-09-06 20:51:47|comment-on-answer|
|     4642|1324220|2008-09-06 21:15:46|comment-on-answer|
+---------+-------+-------------------+-----------------+
only showing top 5 rows



In [19]:
comments_answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [20]:
comments_questions_sdf.count()

20268151

In [21]:
comments_questions_sdf.show(5)

+---------+-------+-------------------+-------------------+
|from_node|to_node|         time_stamp|          edge_type|
+---------+-------+-------------------+-------------------+
|     4550|   4550|2008-09-06 19:26:30|comment-on-question|
|      242|    184|2008-09-06 20:38:23|comment-on-question|
|     4213|   4946|2008-09-07 06:15:49|comment-on-question|
|       91|     91|2008-09-07 06:18:15|comment-on-question|
|     2658|   1874|2008-09-07 07:18:11|comment-on-question|
+---------+-------+-------------------+-------------------+
only showing top 5 rows



In [22]:
comments_questions_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [23]:
graph_sdf.count()

63497050

In [24]:
graph_sdf.show(5)

+---------+-------+-------------------+---------+
|from_node|to_node|         time_stamp|edge_type|
+---------+-------+-------------------+---------+
|        9|      8|2008-08-01 05:17:57|   answer|
|        1|      1|2008-08-01 06:56:41|   answer|
|       13|      1|2008-08-01 15:57:27|   answer|
|       17|      1|2008-08-01 19:07:19|   answer|
|       48|      2|2008-08-01 19:16:22|   answer|
+---------+-------+-------------------+---------+
only showing top 5 rows



In [25]:
graph_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- edge_type: string (nullable = false)



## Step 3

In [26]:
graph_sdf.createOrReplaceTempView('graph_sdf_view')
X=spark.sql('SELECT to_node, COUNT(*) AS indegree FROM graph_sdf_view GROUP BY to_node ORDER BY COUNT(*) DESC LIMIT 5')
X.show()

+-------+--------+
|to_node|indegree|
+-------+--------+
|  22656|   93143|
| 548225|   54410|
|  29407|   45824|
| 157882|   39576|
|1144035|   38909|
+-------+--------+



In [27]:
graph_sdf.createOrReplaceTempView('graph_sdf_view')
Y=spark.sql('SELECT from_node, COUNT(*) AS outdegree FROM graph_sdf_view GROUP BY from_node ORDER BY COUNT(*) DESC LIMIT 5')
Y.show()

+---------+---------+
|from_node|outdegree|
+---------+---------+
|    22656|   101663|
|   505088|    65522|
|    17034|    65260|
|  1491895|    62746|
|   100297|    54330|
+---------+---------+



In [51]:
def spark_bfs(G, origins, max_depth):
    schema = StructType([StructField("node", IntegerType(), True),StructField("depth", IntegerType(), True)])
    frontier = spark.createDataFrame(origins, schema)
    visited= spark.createDataFrame(origins, schema).fillna(0)
    G.repartition(100, 'from_node')
    G.cache()
    for i in range(max_depth):
        #print (i,frontier.count(),visited.count())
        next_nodes=get_nodes(G,frontier, visited, i+1)
        #print (next_nodes.count())
        if i>0:
            visited=visited.unionAll(frontier)
        frontier=next_nodes
        frontier.cache()
        visited.cache()
#     visited=visited.distinct()
    visited=visited.unionAll(next_nodes)
    return visited

In [52]:
def get_nodes(G,curr_nodes, visited, depth):
    G.createOrReplaceTempView('graph_sdf_view')
    curr_nodes.createOrReplaceTempView('frontier_sdf_view')
    visited.createOrReplaceTempView('visited_sdf_view')
    connected_nodes=('SELECT g.to_node as node, CAST(' +str(depth)+ ' AS int )as depth FROM graph_sdf_view AS g INNER JOIN frontier_sdf_view AS f ON f.node=g.from_node')
    next_nodes=spark.sql(connected_nodes)
    next_nodes.createOrReplaceTempView('next_nodes_sdf_view')
    remove_frontier=('SELECT n.node, n.depth FROM next_nodes_sdf_view n LEFT JOIN frontier_sdf_view v ON v.node=n.node WHERE v.node IS NULL')
    next_nodes=spark.sql(remove_frontier)
    next_nodes.createOrReplaceTempView('next_nodes_sdf_view')
    remove_visited=('SELECT n.node, n.depth FROM next_nodes_sdf_view n LEFT JOIN visited_sdf_view v ON v.node=n.node WHERE v.node IS NULL')
    hop_nodes=spark.sql(remove_visited)
    return hop_nodes

## Step 4.1

In [53]:
bfs_sdf=spark_bfs(comments_questions_sdf, [{'node': 4550}, {'node': 242}], 2)

In [54]:
bfs_sdf.count()

397

In [55]:
bfs_sdf[bfs_sdf.depth == 1 ].count()

28

In [56]:
bfs_sdf.createOrReplaceTempView('bfs_sdf_view')
get_nodes=('SELECT node,depth from bfs_sdf_view WHERE depth=2 GROUP BY node,depth HAVING count(*)>1')
filtered_bfs_sdf=spark.sql(get_nodes)

In [57]:
filtered_bfs_sdf.count()

43

In [58]:
def friend_rec (filtered_bfs_sdf , graph_sdf):
    graph_sdf.cache()
    filtered_bfs_sdf.createOrReplaceTempView('filtered_bfs_sdf_view')
    combinations=('SELECT f1.node as n1,f2.node as n2 FROM filtered_bfs_sdf_view f1 INNER JOIN filtered_bfs_sdf_view f2 ON f1.depth=f2.depth WHERE f1.node<>f2.node')
    temp=spark.sql(combinations)
    A=graph_sdf.select('from_node','to_node')
    B=graph_sdf.select('to_node','from_node')
    temp=temp.subtract(A)
    temp=temp.subtract(B)
    return temp

## Step 4.2

In [36]:
recommendations=friend_rec (filtered_bfs_sdf , comments_questions_sdf)

In [37]:
recommendations.count()

1804

In [38]:
recommendations.show()

+-------+-------+
|     n1|     n2|
+-------+-------+
|  21918| 453447|
|  21918|4204628|
|  46646| 267679|
|  59017|1187554|
| 104015|  55503|
| 211452|  59017|
| 503032|2269511|
|1406214| 503032|
|2269511|  21918|
|3830157| 132640|
|4204628|  47550|
|  46646|1127460|
|  55503| 104015|
|  60602|1187554|
| 267679|1061543|
| 393268| 318811|
|1218595|1187554|
|1218595|4800193|
|  27483|  46646|
|  63775|  46646|
+-------+-------+
only showing top 20 rows

